
## Sweeps: An Overview

Running a hyperparameter sweep with Weights & Biases is very easy. There are just 3 simple steps:

1. **Define the sweep:** we do this by creating a dictionary-like object that specifies the sweep: which parameters to search through, which search strategy to use, which metric to optimize.

2. **Initialize the sweep:** with one line of code we initialize the sweep and pass in the dictionary of sweep configurations:
`sweep_id = wandb.sweep(sweep_config)`

3. **Run the sweep agent:** also accomplished with one line of code, we call w`andb.agent()` and pass the `sweep_id` along with a function that defines your model architecture and trains it:
`wandb.agent(sweep_id, function=train)`

And voila! That's all there is to running a hyperparameter sweep!

In the notebook below, we'll walk through these 3 steps in more detail.

We highly encourage you to fork this notebook, tweak the parameters, or try the model with your own dataset!

### Resources
- [Sweeps docs →](https://docs.wandb.com/library/sweeps)
- [Launching from the command line →](https://www.wandb.com/articles/hyperparameter-tuning-as-easy-as-1-2-3)


In [ ]:
#!pip install wandb
import os
import wandb
!wandb login

## 1. Define the Sweep

Weights & Biases sweeps give you powerful levers to configure your sweeps exactly how you want them, with just a few lines of code. The sweeps config can be defined as
[a dictionary or a YAML file](https://docs.wandb.ai/guides/sweeps/configuration).

Let's walk through some of them together:
*   **Metric** – This is the metric the sweeps are attempting to optimize. Metrics can take a `name` (this metric should be logged by your training script) and a `goal` (`maximize` or `minimize`). 
*   **Search Strategy** – Specified using the `"method"` key. We support several different search strategies with sweeps. 
  *   **Grid Search** – Iterates over every combination of hyperparameter values.
  *   **Random Search** – Iterates over randomly chosen combinations of hyperparameter values.
  *   **Bayesian Search** – Creates a probabilistic model that maps hyperparameters to probability of a metric score, and chooses parameters with high probability of improving the metric. The objective of Bayesian optimization is to spend more time in picking the hyperparameter values, but in doing so trying out fewer hyperparameter values.
*   **Parameters** – A dictionary containing the hyperparameter names, and discrete values, a range, or distributions from which to pull their values on each iteration.

You can find a list of all configuration options [here](https://docs.wandb.com/library/sweeps/configuration).

In [ ]:
os.environ["WANDB_NOTEBOOK_NAME"] = "Ao_XGBoost_Sweep.ipynb"

sweep_config = {
    "method": "random", # try grid or random
    "metric": {
      "name": "error",
      "goal": "minimize"   
    },
    "parameters": {
        "booster": {
            "values": ["gbtree"]
        },
        "max_depth": {
      "min": 6,
      "max": 10
        },
#    "learning_rate" :{
 #     "distribution": "log_uniform",
 #     "min": -4,
 #     "max": -2
 #       },
    "learning_rate" :{
        "values": [0.04954]
        },        
    "subsample": {
        "values": [0.4372]
    },
    "colsample_bynode": {
        "values": [0.7597]
    },    
    "n_estimators": {
        "values": [3000, 10000, 20000]
    },
    "reg_alpha": {
        "values": [0.1668]
    },    
    "reg_lambda": {
        "values": [0.5222]
    }                
    }
}

## 2. Initialize the Sweep

Calling `wandb.sweep` starts a Sweep Controller --
a centralized process that provides settings of the `parameters` to any who query it
and expects them to return performance on `metrics` via `wandb` logging.

In [ ]:
sweep_id = wandb.sweep(sweep_config, project="XGBoost-sweeps")

### Define your training process
Before we can run the sweep,
we need to define a function that creates and trains the model --
the function that takes in hyperparameter values and spits out metrics.

We'll also need `wandb` to be integrated into our script.
There's three main components:
*   `wandb.init()` – Initialize a new W&B run. Each run is single execution of the training script.
*   `wandb.config` – Save all your hyperparameters in a config object. This lets you use [our app](https://wandb.ai) to sort and compare your runs by hyperparameter values.
*   `wandb.log()` – Logs metrics and custom objects – these can be images, videos, audio files, HTML, plots, point clouds etc.

We also need to download the data:

In [ ]:
# XGBoost model for Pima Indians dataset
from numpy import loadtxt
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pyarrow.feather as feather
from sklearn.multioutput import MultiOutputRegressor
import numpy as np
import math



In [ ]:
fluxData_df = feather.read_feather('data/fluxData.feather')
zernikeData_df = feather.read_feather('data/zernikeData.feather')

In [ ]:

def train():
  config_defaults = {
    "booster": "gbtree",
    "max_depth": 3,
    "learning_rate": 0.1,
    "subsample": 1,
    "seed": 117,
    "test_size": 0.33,
  }

  wandb.init(config=config_defaults)  # defaults are over-ridden during the sweep
  config = wandb.config

  # load data and split into predictors and targets
  X, Y = fluxData_df, zernikeData_df

  # split data into train and test sets
  X_train, X_test, y_train, y_test = train_test_split(X, Y,
                                                      test_size=config.test_size,
                                                      random_state=config.seed)

  # fit model on train
# fitting
  result = MultiOutputRegressor(xgb.XGBRegressor(objective='reg:squarederror', colsample_bynode = config.colsample_bynode, max_depth = config.max_depth, reg_alpha=config.reg_alpha, reg_lambda = config.reg_lambda, subsample = config.subsample,
  n_estimators = config.n_estimators, learning_rate = config.learning_rate, tree_method ="hist", booster =config.booster)).fit(X_train, y_train)
  predictResult = result.predict(X_test)
  predictError =predictResult-y_test
  tmp = predictError.to_numpy().flatten()
  error=np.sqrt(np.sum(tmp**2)/tmp.size)
  wandb.log({"error": error})

## 3. Run the Sweep with an agent

Now, we call `wandb.agent` to start up our sweep.

You can call `wandb.agent` on any machine where you're logged into W&B that has
- the `sweep_id`,
- the dataset and `train` function

and that machine will join the sweep!

> _Note_: a `random` sweep will by defauly run forever,
trying new parameter combinations until the cows come home --
or until you [turn the sweep off from the app UI](https://docs.wandb.ai/ref/app/features/sweeps).
You can prevent this by providing the total `count` of runs you'd like the `agent` to complete.

In [8]:
wandb.agent(sweep_id, train, count=2500)